## **Import Library & Load Data**

In [23]:
total_transaction_amount = df['Transaction Amount'].sum()
print(f"Total Transaction Amount: {total_transaction_amount}")

Total Transaction Amount: 1425377511.1899993


In [1]:
# IMPORT LIBRARY

import pandas as pd
import numpy as np

In [2]:
# Memuat dataset transaksi bank
df = pd.read_csv("bank_transactions.csv")

# Melihat struktur awal dataset
print("INFO AWAL DATASET")
df.info()

INFO AWAL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [3]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T33671,C1010011,8/19/1992,F,NOIDA,32500.73,9/26/2016,123813,4750.0
1,T89544,C1010014,6/4/1992,F,MUMBAI,38377.14,8/1/2016,154451,1205.0
2,T971994,C1010018,5/29/1990,F,CHAMPARAN,496.18,9/15/2016,170254,30.0
3,T642232,C1010028,8/25/1988,F,DELHI,296828.37,8/29/2016,95212,557.0
4,T898045,C1010038,7/13/1992,F,LOHIT,1290.76,9/7/2016,112425,100.0


## **Konversi Tipe Data**

Pada tahap ini dilakukan penyesuaian tipe data agar sesuai dengan konteks analisis.

- Kolom tanggal dikonversi ke `datetime`

- Kolom kategorikal dikonversi ke `category`

In [4]:
# Konversi kolom tanggal
df["CustomerDOB"] = pd.to_datetime(df["CustomerDOB"], errors="coerce")
df["TransactionDate"] = pd.to_datetime(df["TransactionDate"], errors="coerce")

# Konversi kolom kategorikal
df["CustGender"] = df["CustGender"].astype("category")
df["CustLocation"] = df["CustLocation"].astype("category")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1048567 non-null  object        
 1   CustomerID               1048567 non-null  object        
 2   CustomerDOB              1045170 non-null  datetime64[ns]
 3   CustGender               1047467 non-null  category      
 4   CustLocation             1048416 non-null  category      
 5   CustAccountBalance       1046198 non-null  float64       
 6   TransactionDate          1048567 non-null  datetime64[ns]
 7   TransactionTime          1048567 non-null  int64         
 8   TransactionAmount (INR)  1048567 non-null  float64       
dtypes: category(2), datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 59.3+ MB


## **Handling Missing Values**

Strategi penanganan missing values yang digunakan:

- **Account Balance** → median (lebih robust terhadap outlier)

- **Gender** → modus

- **Location** → modus

In [5]:
# Imputasi CustAccountBalance dengan median
median_balance = df["CustAccountBalance"].median()
df["CustAccountBalance"] = df["CustAccountBalance"].fillna(median_balance)

# Imputasi CustGender dengan modus
mode_gender = df["CustGender"].mode()[0]
df["CustGender"] = df["CustGender"].fillna(mode_gender)

# Imputasi CustLocation dengan modus
mode_location = df["CustLocation"].mode()[0]
df["CustLocation"] = df["CustLocation"].fillna(mode_location)

print("Missing values setelah imputasi:")
print(df.isnull().sum())


Missing values setelah imputasi:
TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                    0
CustLocation                  0
CustAccountBalance            0
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64


## **Validasi & Pembersihan Customer Date of Birth**

Tanggal lahir di luar rentang realistis dianggap tidak valid.

- Rentang DOB valid: **1929 – 2015**
- Nilai di luar rentang → `NaT`
- Baris tanpa DOB kemudian dihapus

In [6]:
lower_bound = pd.to_datetime("1929-01-01")
upper_bound = pd.to_datetime("2015-12-31")

# Tandai DOB tidak realistis sebagai NaT
df.loc[
    (df["CustomerDOB"] < lower_bound) |
    (df["CustomerDOB"] > upper_bound),
    "CustomerDOB"
] = pd.NaT

# Drop baris tanpa DOB
df = df.dropna(subset=["CustomerDOB"])


In [7]:
print("Missing values setelah imputasi:")
print(df.isnull().sum())

Missing values setelah imputasi:
TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64


## **Data Quality Tambahan**

### Standarisasi Gender

Nilai gender tidak valid (`T`) disamakan menjadi `M`.

In [8]:
print("Unique values and their frequencies for 'CustGender':")
print(df['CustGender'].value_counts())

Unique values and their frequencies for 'CustGender':
CustGender
M    712217
F    272754
T         0
Name: count, dtype: int64


In [9]:
df['CustGender'] = df['CustGender'].replace('T', 'M')

print("Unique values and their frequencies for 'CustGender' after replacement:")
print(df['CustGender'].value_counts())

Unique values and their frequencies for 'CustGender' after replacement:
CustGender
M    712217
F    272754
Name: count, dtype: int64


/tmp/ipython-input-2520148409.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['CustGender'] = df['CustGender'].replace('T', 'M')


### Standarisasi Nama Kota (Fuzzy Matching)

Kolom CustLocation memiliki ribuan variasi penulisan (typo, singkatan, dll).
Tujuan tahap ini:

- Mengurangi ribuan nilai unik
- Menyatukan nama kota yang mirip
- Menyimpan kota besar
- Mengelompokkan sisanya sebagai OTHER CITY

In [10]:
print("\nUnique values and their frequencies for 'CustLocation':")
print(df['CustLocation'].value_counts())


Unique values and their frequencies for 'CustLocation':
CustLocation
MUMBAI                               97973
BANGALORE                            77731
NEW DELHI                            76141
GURGAON                              70422
DELHI                                67297
                                     ...  
STUDIO ANDHERI WEST MUMBAI MUMBAI        0
STREET MUMBAI                            0
(BEFORE YMCA BLDG) CHENNAI               0
(MCORP) 1/C NR HOTEL GRANT MUMBAI        0
YASHASHRI HOSPITAL PUNE                  0
Name: count, Length: 9355, dtype: int64


In [11]:
!pip install rapidfuzz
from rapidfuzz import process, fuzz

# Ambil semua city unique
raw_city = (
    df["CustLocation"]
    .astype(str)
    .str.upper()
    .str.strip()
)

# Ambil Top 250 kota dari dataset (data-driven)
top_cities = raw_city.value_counts().head(250).index.tolist()

# Mapping fuzzy
mapping = {}
unique_vals = raw_city.unique()

for raw in unique_vals:
    match, score, _ = process.extractOne(raw, top_cities, scorer=fuzz.WRatio)
    mapping[raw] = match if score >= 85 else "OTHER CITY"

# Apply
df["CustLocation"] = raw_city.map(mapping)

print("Final unique:", df["CustLocation"].nunique())
print(df["CustLocation"].value_counts().head(20))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.1 MB/s eta 0:00:00
Final unique: 251
CustLocation
MUMBAI         110957
BANGALORE       84898
NEW DELHI       77296
OTHER CITY      74627
GURGAON         71119
DELHI           68325
NOIDA           31960
CHENNAI         28432
PUNE            28284
THANE           23766
HYDERABAD       23398
KOLKATA         19491
GHAZIABAD       14744
NAVI MUMBAI     12144
AHMEDABAD       11509
FARIDABAD       10880
JAIPUR           9481
CHANDIGARH       8803
LUCKNOW          7329
MOHALI           6065
Name: count, dtype: int64


### Hapus Baris Duplikat

In [12]:
print(f"Number of duplicate rows before removal: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Number of duplicate rows after removal: {df.duplicated().sum()}")
print(f"New DataFrame shape after duplicate removal: {df.shape}")

Number of duplicate rows before removal: 0
Number of duplicate rows after removal: 0
New DataFrame shape after duplicate removal: (984971, 9)


## **Menghapus Transaksi dengan Nilai Negatif**

Transaksi dengan nilai `<= 0` dianggap tidak logis untuk analisis umum.

In [13]:
print(f"Jumlah transaksi bernilai negatif sebelum difilter: {(df['TransactionAmount (INR)'] <= 0).sum()}")
print(f"Ukuran DataFrame sebelum filtering: {df.shape}")

Jumlah transaksi bernilai negatif sebelum difilter: 712
Ukuran DataFrame sebelum filtering: (984971, 9)


In [14]:
df = df[df["TransactionAmount (INR)"] > 0]

In [15]:
print(f"Jumlah transaksi bernilai negatif setelah difilter: {(df['TransactionAmount (INR)'] < 0).sum()}")
print(f"Ukuran DataFrame setelah filtering: {df.shape}")


Jumlah transaksi bernilai negatif setelah difilter: 0
Ukuran DataFrame setelah filtering: (984259, 9)


## **Menghapus Kolom yang Tidak Digunakan**

Kolom berikut dihapus karena tidak relevan untuk analisis lanjutan:

- `TransactionID`
- `TransactionTime`

In [16]:
df = df.drop(columns=[
    "TransactionID",
    "TransactionTime"
])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984259 entries, 0 to 1048560
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CustomerID               984259 non-null  object        
 1   CustomerDOB              984259 non-null  datetime64[ns]
 2   CustGender               984259 non-null  category      
 3   CustLocation             984259 non-null  object        
 4   CustAccountBalance       984259 non-null  float64       
 5   TransactionDate          984259 non-null  datetime64[ns]
 6   TransactionAmount (INR)  984259 non-null  float64       
dtypes: category(1), datetime64[ns](2), float64(2), object(2)
memory usage: 53.5+ MB


## **Rename Kolom (Standarisasi Nama)**

Nama kolom disesuaikan agar lebih readable dan konsisten untuk analisis lebih lanjut.

In [18]:
df = df.rename(columns={
    "CustomerID": "Customer ID",
    "CustomerDOB": "DOB",
    "CustGender": "Gender",
    "CustLocation": "City/District",
    "CustAccountBalance": "Account Balance",
    "TransactionDate": "Transaction Date",
    "TransactionAmount (INR)": "Transaction Amount"
})

## **Final Check & Export Data**

In [19]:
print("INFO DATASET AKHIR")
df.info()

INFO DATASET AKHIR
<class 'pandas.core.frame.DataFrame'>
Index: 984259 entries, 0 to 1048560
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Customer ID         984259 non-null  object        
 1   DOB                 984259 non-null  datetime64[ns]
 2   Gender              984259 non-null  category      
 3   City/District       984259 non-null  object        
 4   Account Balance     984259 non-null  float64       
 5   Transaction Date    984259 non-null  datetime64[ns]
 6   Transaction Amount  984259 non-null  float64       
dtypes: category(1), datetime64[ns](2), float64(2), object(2)
memory usage: 53.5+ MB
